데이터 불러오기

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv("/home/t5user3/아파트실거래가/train.csv")
test = pd.read_csv("/home/t5user3/아파트실거래가/test.csv")
submission = pd.read_csv("/home/t5user3/아파트실거래가/sample_submission.csv")
interest_rate = pd.read_csv("/home/t5user3/아파트실거래가/한국은행 기준금리 및 여수신금리.csv",encoding='cp949')
#한국은행 금리데이터 링크 (https://ecos.bok.or.kr/#/SearchStat) >1.3.1. 한국은행 기준금리 및 여수신금리 > 한국은행 기준금리 다운로드

In [2]:
train.head()

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion
0,TRAIN_0000,서울특별시 강남구 대치동,503,개포우성1,158.54,201401,7,174000,13,1983
1,TRAIN_0001,서울특별시 강남구 대치동,503,개포우성1,127.61,201401,9,157500,6,1983
2,TRAIN_0002,서울특별시 강남구 대치동,503,개포우성1,127.61,201401,19,150000,5,1983
3,TRAIN_0003,서울특별시 강남구 대치동,503,개포우성1,127.61,201402,10,152000,9,1983
4,TRAIN_0004,서울특별시 강남구 대치동,503,개포우성1,84.81,201402,10,116000,3,1983


In [3]:
test.head()

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,floor,year_of_completion
0,TEST_0000,서울특별시 강남구 대치동,977,(977-),77.97,202304,10,2,2021
1,TEST_0001,서울특별시 강남구 대치동,977,(977-),59.99,202304,10,2,2021
2,TEST_0002,서울특별시 강남구 대치동,503,개포우성1,84.81,202301,13,3,1983
3,TEST_0003,서울특별시 강남구 대치동,503,개포우성1,84.81,202301,19,9,1983
4,TEST_0004,서울특별시 강남구 대치동,503,개포우성1,84.81,202302,15,5,1983


In [4]:
submission.head()

,id,transaction_real_price
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0


In [5]:
interest_rate.head()

,통계표,계정항목,단위,변환,2014.01,2014.02,2014.03,2014.04,2014.05,2014.06,...,2022.09,2022.10,2022.11,2022.12,2023.01,2023.02,2023.03,2023.04,2023.05,2023.06
0,1.3.1. 한국은행 기준금리 및 여수신금리,한국은행 기준금리,연%,원자료,2.5,2.5,2.5,2.5,2.5,2.5,...,2.5,3,3.25,3.25,3.5,3.5,3.5,3.5,3.5,3.5


금리 데이터 추가 (한국은행 기준 금리)

In [6]:
# Define the replace_date function
def replace_date(date_str):
       return date_str.upper()

# Now you can use the replace_date function in your code
interest_rate.drop(['통계표','단위','계정항목','변환'], axis = 1, inplace = True)
interest = interest_rate.T.reset_index()
interest['index'] = interest['index'].apply(replace_date)
interest.columns = ['index', 'interest']
interest.head()


,index,interest
0,2014.01,2.5
1,2014.02,2.5
2,2014.03,2.5
3,2014.04,2.5
4,2014.05,2.5


In [7]:
# Define the function to convert the date format
def convert_to_yyyymm(date_str):
    year_month_list = date_str.split('.')
    year = year_month_list[0]
    month = year_month_list[1]
    return year + month

# Apply the function to the 'index' column
interest['index'] = interest['index'].apply(convert_to_yyyymm)


interest['index'] = interest['index'].astype(int)

print(interest)

      index  interest
0    201401       2.5
1    201402       2.5
2    201403       2.5
3    201404       2.5
4    201405       2.5
..      ...       ...
109  202302       3.5
110  202303       3.5
111  202304       3.5
112  202305       3.5
113  202306       3.5

[114 rows x 2 columns]


In [8]:
train = pd.merge(train, interest, left_on='transaction_year_month', right_on='index', how='left')


In [9]:
train.head(3)

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,index,interest
0,TRAIN_0000,서울특별시 강남구 대치동,503,개포우성1,158.54,201401,7,174000,13,1983,201401,2.5
1,TRAIN_0001,서울특별시 강남구 대치동,503,개포우성1,127.61,201401,9,157500,6,1983,201401,2.5
2,TRAIN_0002,서울특별시 강남구 대치동,503,개포우성1,127.61,201401,19,150000,5,1983,201401,2.5


In [10]:
test = pd.merge(test, interest, left_on='transaction_year_month', right_on='index', how='left')

In [11]:
test.head(3)

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,floor,year_of_completion,index,interest
0,TEST_0000,서울특별시 강남구 대치동,977,(977-),77.97,202304,10,2,2021,202304,3.5
1,TEST_0001,서울특별시 강남구 대치동,977,(977-),59.99,202304,10,2,2021,202304,3.5
2,TEST_0002,서울특별시 강남구 대치동,503,개포우성1,84.81,202301,13,3,1983,202301,3.5


날짜 데이터 분리(년, 월)

In [12]:
#날짜 데이터 분리
train['transaction_year'] = train['transaction_year_month'].apply(lambda x : int(str(x)[:4]))
train['transaction_month'] = train['transaction_year_month'].apply(lambda x : int(str(x)[4:]))

In [13]:
train.head(3)

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,index,interest,transaction_year,transaction_month
0,TRAIN_0000,서울특별시 강남구 대치동,503,개포우성1,158.54,201401,7,174000,13,1983,201401,2.5,2014,1
1,TRAIN_0001,서울특별시 강남구 대치동,503,개포우성1,127.61,201401,9,157500,6,1983,201401,2.5,2014,1
2,TRAIN_0002,서울특별시 강남구 대치동,503,개포우성1,127.61,201401,19,150000,5,1983,201401,2.5,2014,1


In [14]:
#날짜 데이터 분리
test['transaction_year'] = test['transaction_year_month'].apply(lambda x : int(str(x)[:4]))
test['transaction_month'] = test['transaction_year_month'].apply(lambda x : int(str(x)[4:]))

In [15]:
test.head()

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,floor,year_of_completion,index,interest,transaction_year,transaction_month
0,TEST_0000,서울특별시 강남구 대치동,977,(977-),77.97,202304,10,2,2021,202304,3.5,2023,4
1,TEST_0001,서울특별시 강남구 대치동,977,(977-),59.99,202304,10,2,2021,202304,3.5,2023,4
2,TEST_0002,서울특별시 강남구 대치동,503,개포우성1,84.81,202301,13,3,1983,202301,3.5,2023,1
3,TEST_0003,서울특별시 강남구 대치동,503,개포우성1,84.81,202301,19,9,1983,202301,3.5,2023,1
4,TEST_0004,서울특별시 강남구 대치동,503,개포우성1,84.81,202302,15,5,1983,202302,3.5,2023,2


In [16]:
train['주소'] = train['sigungu'] + train['jibun']

print(train)

              id        sigungu jibun apt_name  exclusive_use_area  \
0     TRAIN_0000  서울특별시 강남구 대치동   503    개포우성1              158.54   
1     TRAIN_0001  서울특별시 강남구 대치동   503    개포우성1              127.61   
2     TRAIN_0002  서울특별시 강남구 대치동   503    개포우성1              127.61   
3     TRAIN_0003  서울특별시 강남구 대치동   503    개포우성1              127.61   
4     TRAIN_0004  서울특별시 강남구 대치동   503    개포우성1               84.81   
...          ...            ...   ...      ...                 ...   
5982  TRAIN_5982  서울특별시 강남구 대치동   511  한보미도맨션2              190.47   
5983  TRAIN_5983  서울특별시 강남구 대치동   511  한보미도맨션2              126.33   
5984  TRAIN_5984  서울특별시 강남구 대치동   511  한보미도맨션2              126.33   
5985  TRAIN_5985  서울특별시 강남구 대치동   511  한보미도맨션2              126.33   
5986  TRAIN_5986  서울특별시 강남구 대치동   992      현대1               84.27   

      transaction_year_month  transaction_day  transaction_real_price  floor  \
0                     201401                7                  174000     13   

In [17]:
test['주소'] = test['sigungu'] + test['jibun']

불필요 데이터 정리

In [18]:
train = train[['주소','transaction_year','transaction_month' ,'exclusive_use_area','floor', 'year_of_completion','interest','transaction_real_price']]
test = test[['주소','transaction_year','transaction_month' ,'exclusive_use_area','floor', 'year_of_completion','interest']]

In [19]:
train

,주소,transaction_year,transaction_month,exclusive_use_area,floor,year_of_completion,interest,transaction_real_price
0,서울특별시 강남구 대치동503,2014,1,158.54,13,1983,2.50,174000
1,서울특별시 강남구 대치동503,2014,1,127.61,6,1983,2.50,157500
2,서울특별시 강남구 대치동503,2014,1,127.61,5,1983,2.50,150000
3,서울특별시 강남구 대치동503,2014,2,127.61,9,1983,2.50,152000
4,서울특별시 강남구 대치동503,2014,2,84.81,3,1983,2.50,116000
...,...,...,...,...,...,...,...,...
5982,서울특별시 강남구 대치동511,2022,5,190.47,8,1985,1.75,507500
5983,서울특별시 강남구 대치동511,2022,6,126.33,10,1985,1.75,380000
5984,서울특별시 강남구 대치동511,2022,6,126.33,10,1985,1.75,380000
5985,서울특별시 강남구 대치동511,2022,7,126.33,7,1985,2.25,380000


In [20]:
test

,주소,transaction_year,transaction_month,exclusive_use_area,floor,year_of_completion,interest
0,서울특별시 강남구 대치동977,2023,4,77.97,2,2021,3.5
1,서울특별시 강남구 대치동977,2023,4,59.99,2,2021,3.5
2,서울특별시 강남구 대치동503,2023,1,84.81,3,1983,3.5
3,서울특별시 강남구 대치동503,2023,1,84.81,9,1983,3.5
4,서울특별시 강남구 대치동503,2023,2,84.81,5,1983,3.5
...,...,...,...,...,...,...,...
191,서울특별시 강남구 대치동511,2023,6,84.96,1,1985,3.5
192,서울특별시 강남구 대치동511,2023,6,126.33,1,1985,3.5
193,서울특별시 강남구 대치동511,2023,6,190.47,13,1985,3.5
194,서울특별시 강남구 대치동511,2023,6,126.33,12,1985,3.5


주소 >> 위도,경도로 변환 (Geocoding - NAVER API)

In [21]:
import requests
import pandas as pd

def geocode_naver(address):
    client_id = 'o4oebhsmqx'
    client_secret = 'SYvoBghZEE2nomTpYHPISGv23XPD8UvgWVR64til'
    base_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'

    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret
    }

    params = {
        'query': address
    }

    response = requests.get(base_url, headers=headers, params=params)
    data = response.json()

    if data.get('status') == 'OK':
        result = data.get('addresses')[0]
        return result['y'], result['x']  # 위도와 경도 반환
    else:
        return None, None




# Create new columns for latitude ('위도') and longitude ('경도')
train['위도'] = None
train['경도'] = None

# Define a function to geocode each address and update the DataFrame
def geocode_and_update(row):
    latitude, longitude = geocode_naver(row['주소'])
    row['위도'] = latitude
    row['경도'] = longitude
    return row

# Apply the geocode_and_update function to each row in the DataFrame
train = train.apply(geocode_and_update, axis=1)

print(train)

                    주소  transaction_year  transaction_month  \
0     서울특별시 강남구 대치동503              2014                  1   
1     서울특별시 강남구 대치동503              2014                  1   
2     서울특별시 강남구 대치동503              2014                  1   
3     서울특별시 강남구 대치동503              2014                  2   
4     서울특별시 강남구 대치동503              2014                  2   
...                ...               ...                ...   
5982  서울특별시 강남구 대치동511              2022                  5   
5983  서울특별시 강남구 대치동511              2022                  6   
5984  서울특별시 강남구 대치동511              2022                  6   
5985  서울특별시 강남구 대치동511              2022                  7   
5986  서울특별시 강남구 대치동992              2022                  8   

      exclusive_use_area  floor  year_of_completion  interest  \
0                 158.54     13                1983      2.50   
1                 127.61      6                1983      2.50   
2                 127.61      5                1

In [22]:
import requests

def geocode_naver(address):
    client_id = 'o4oebhsmqx'
    client_secret = 'SYvoBghZEE2nomTpYHPISGv23XPD8UvgWVR64til'
    base_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'

    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret
    }

    params = {
        'query': address
    }

    response = requests.get(base_url, headers=headers, params=params)
    data = response.json()

    if data.get('status') == 'OK':
        result = data.get('addresses')[0]
        return result['y'], result['x']  # 위도와 경도 반환
    else:
        return None, None



# Create new columns for latitude ('위도') and longitude ('경도')
test['위도'] = None
test['경도'] = None

# Define a function to geocode each address and update the DataFrame
def geocode_and_update(row):
    latitude, longitude = geocode_naver(row['주소'])
    row['위도'] = latitude
    row['경도'] = longitude
    return row

# Apply the geocode_and_update function to each row in the DataFrame
test= test.apply(geocode_and_update, axis=1)

print(test)

                   주소  transaction_year  transaction_month  \
0    서울특별시 강남구 대치동977              2023                  4   
1    서울특별시 강남구 대치동977              2023                  4   
2    서울특별시 강남구 대치동503              2023                  1   
3    서울특별시 강남구 대치동503              2023                  1   
4    서울특별시 강남구 대치동503              2023                  2   
..                ...               ...                ...   
191  서울특별시 강남구 대치동511              2023                  6   
192  서울특별시 강남구 대치동511              2023                  6   
193  서울특별시 강남구 대치동511              2023                  6   
194  서울특별시 강남구 대치동511              2023                  6   
195  서울특별시 강남구 대치동992              2023                  3   

     exclusive_use_area  floor  year_of_completion  interest          위도  \
0                 77.97      2                2021       3.5  37.5019370   
1                 59.99      2                2021       3.5  37.5019370   
2                 84.81    

In [23]:
# '위도(Latitude)'와 '경도(Longitude)' 열을 수치형으로 변환
train['위도'] = pd.to_numeric(train['위도'])
train['경도'] = pd.to_numeric(train['경도'])
test['위도'] = pd.to_numeric(test['위도'])
test['경도'] = pd.to_numeric(test['경도'])


In [24]:
print(train.dtypes)
print(test.dtypes)

주소                         object
transaction_year            int64
transaction_month           int64
exclusive_use_area        float64
floor                       int64
year_of_completion          int64
interest                  float64
transaction_real_price      int64
위도                        float64
경도                        float64
dtype: object
주소                     object
transaction_year        int64
transaction_month       int64
exclusive_use_area    float64
floor                   int64
year_of_completion      int64
interest              float64
위도                    float64
경도                    float64
dtype: object


학습에 사용할 데이터

In [72]:
train = train[['transaction_year','transaction_month' ,'exclusive_use_area','floor', 'year_of_completion','interest','위도','경도','transaction_real_price']]
test = test[['transaction_year','transaction_month' ,'exclusive_use_area','floor', 'year_of_completion','interest','위도','경도']]

In [73]:
train.head(3)

,transaction_year,transaction_month,exclusive_use_area,floor,year_of_completion,interest,위도,경도,transaction_real_price
0,2014,1,158.54,13,1983,2.5,37.49112,127.059897,174000
1,2014,1,127.61,6,1983,2.5,37.49112,127.059897,157500
2,2014,1,127.61,5,1983,2.5,37.49112,127.059897,150000


In [74]:
test.head(3)

,transaction_year,transaction_month,exclusive_use_area,floor,year_of_completion,interest,위도,경도
0,2023,4,77.97,2,2021,3.5,37.501937,127.064819
1,2023,4,59.99,2,2021,3.5,37.501937,127.064819
2,2023,1,84.81,3,1983,3.5,37.491120,127.059897


하이퍼 파라미터 튜닝(그리드 서치)

In [75]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# 데이터를 훈련과 검증으로 나눕니다.
train_x, val_x, train_y, val_y = train_test_split(train.drop('transaction_real_price', axis=1),
                                                  train['transaction_real_price'], test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
# 랜덤 포레스트, GBM, Extra Trees 의 하이퍼파라미터 그리드
rf_param_grid = {
    'n_estimators': [50, 100, 200, 400],
    'max_depth': [None, 10, 20, 30],
    'max_features': [1.0, "sqrt", "log2"]
}

gbm_param_grid = {
    'n_estimators': [50, 100, 200, 400],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.2, 0.1, 0.01, 0.001],
    'subsample': [1.0, 0.5, 0.25]
}



extra_trees_param_grid = {
    'n_estimators': [50, 100, 200, 400],
    'max_depth': [None, 10, 20, 30],
    'max_features': [1.0, "sqrt", "log2"]
}

# GridSearchCV를 사용하여 최적의 하이퍼파라미터를 탐색
rf_grid_search = GridSearchCV(RandomForestRegressor(random_state=42), rf_param_grid, cv=3)
rf_grid_search.fit(train_x, train_y)

gbm_grid_search = GridSearchCV(GradientBoostingRegressor(random_state=42), gbm_param_grid, cv=3)
gbm_grid_search.fit(train_x, train_y)

extra_trees_grid_search = GridSearchCV(ExtraTreesRegressor(random_state=42), extra_trees_param_grid, cv=3)
extra_trees_grid_search.fit(train_x, train_y)



# 최적의 하이퍼파라미터로 모델 생성 및 훈련
best_rf_model = rf_grid_search.best_estimator_
best_gbm_model = gbm_grid_search.best_estimator_
best_extra_trees_model = extra_trees_grid_search.best_estimator_


# 검증 세트로 예측 수행
rf_predictions = best_rf_model.predict(val_x)
gbm_predictions = best_gbm_model.predict(val_x)
extra_trees_predictions = best_extra_trees_model.predict(val_x)


# 네 모델의 예측 결과를 평균하여 앙상블
ensemble_predictions = (rf_predictions + gbm_predictions + extra_trees_predictions) / 3

MAE값 계산 val_y-prediction 

In [77]:
import numpy as np

def mean_absolute_error(val_y, ensemble_predictions):
    # 실제값과 예측값의 차이를 계산하여 절대값으로 오차 배열 생성
    errors = np.abs(val_y - ensemble_predictions)

    # 오차 배열의 평균 계산 (MAE)
    mae = np.mean(errors)
    return mae

# MAE 계산
mae = mean_absolute_error(val_y, ensemble_predictions)
print("MAE:", mae)

MAE: 5008.9150744134295


In [78]:
train_x = train.drop('transaction_real_price', axis=1)
train_y = train['transaction_real_price']

In [79]:
train_x

,transaction_year,transaction_month,exclusive_use_area,floor,year_of_completion,interest,위도,경도
0,2014,1,158.54,13,1983,2.50,37.491120,127.059897
1,2014,1,127.61,6,1983,2.50,37.491120,127.059897
2,2014,1,127.61,5,1983,2.50,37.491120,127.059897
3,2014,2,127.61,9,1983,2.50,37.491120,127.059897
4,2014,2,84.81,3,1983,2.50,37.491120,127.059897
...,...,...,...,...,...,...,...,...
5982,2022,5,190.47,8,1985,1.75,37.493383,127.067333
5983,2022,6,126.33,10,1985,1.75,37.493383,127.067333
5984,2022,6,126.33,10,1985,1.75,37.493383,127.067333
5985,2022,7,126.33,7,1985,2.25,37.493383,127.067333


In [80]:
train_y

0       174000
1       157500
2       150000
3       152000
4       116000
         ...  
5982    507500
5983    380000
5984    380000
5985    380000
5986    205000
Name: transaction_real_price, Length: 5987, dtype: int64

In [81]:
test

,transaction_year,transaction_month,exclusive_use_area,floor,year_of_completion,interest,위도,경도
0,2023,4,77.97,2,2021,3.5,37.501937,127.064819
1,2023,4,59.99,2,2021,3.5,37.501937,127.064819
2,2023,1,84.81,3,1983,3.5,37.491120,127.059897
3,2023,1,84.81,9,1983,3.5,37.491120,127.059897
4,2023,2,84.81,5,1983,3.5,37.491120,127.059897
...,...,...,...,...,...,...,...,...
191,2023,6,84.96,1,1985,3.5,37.493383,127.067333
192,2023,6,126.33,1,1985,3.5,37.493383,127.067333
193,2023,6,190.47,13,1985,3.5,37.493383,127.067333
194,2023,6,126.33,12,1985,3.5,37.493383,127.067333


In [82]:

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV

# 랜덤 포레스트, GBM, Extra Trees의 하이퍼파라미터 그리드
rf_param_grid = {
    'n_estimators': [50, 100, 200, 400],
    'max_depth': [None, 10, 20, 30],
    'max_features': [1.0, "sqrt", "log2"]
}

gbm_param_grid = {
    'n_estimators': [50, 100, 200, 400],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.2, 0.1, 0.01, 0.001],
    'subsample': [1.0, 0.5, 0.25]
}



extra_trees_param_grid = {
    'n_estimators': [50, 100, 200, 400],
    'max_depth': [None, 10, 20, 30],
    'max_features': [1.0, "sqrt", "log2"]
}

# GridSearchCV를 사용하여 최적의 하이퍼파라미터를 탐색
rf_grid_search = GridSearchCV(RandomForestRegressor(random_state=42), rf_param_grid, cv=3)
rf_grid_search.fit(train_x, train_y)

gbm_grid_search = GridSearchCV(GradientBoostingRegressor(random_state=42), gbm_param_grid, cv=3)
gbm_grid_search.fit(train_x, train_y)

extra_trees_grid_search = GridSearchCV(ExtraTreesRegressor(random_state=42), extra_trees_param_grid, cv=3)
extra_trees_grid_search.fit(train_x, train_y)



# 최적의 하이퍼파라미터로 모델 생성 및 훈련
best_rf_model = rf_grid_search.best_estimator_
best_gbm_model = gbm_grid_search.best_estimator_
best_extra_trees_model = extra_trees_grid_search.best_estimator_


# 검증 세트로 예측 수행
rf_predictions = best_rf_model.predict(test)
gbm_predictions = best_gbm_model.predict(test)
extra_trees_predictions = best_extra_trees_model.predict(test)


#  모델의 예측 결과를 평균하여 앙상블
ensemble_predictions = (rf_predictions + gbm_predictions + extra_trees_predictions) / 3

In [83]:
submission['transaction_real_price'] = ensemble_predictions
submission.head()

,id,transaction_real_price
0,TEST_0000,166461.600566
1,TEST_0001,130904.074684
2,TEST_0002,268678.535970
3,TEST_0003,272625.529331
4,TEST_0004,273387.882155


In [84]:
submission.to_csv('RF+GBM+ET.csv',index = False)